In [1]:
!pip install stable-baselines3
!pip install pyglet
!pip install swig
!pip install box2d-py
!pip install importlib-metadata==4.13.0
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
import os
import gym

from stable_baselines3.common.evaluation import evaluate_policy

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

from typing import Any, Dict
import torch
import torch.nn as nn

from stable_baselines3.common.callbacks import EvalCallback

import torch as th

In [3]:
# #########################
# # OPTIMISER HYPERPARAMS #
# #########################
# env_id = "LunarLander-v2"
# #opti_env = gym.make(env_id)
# eval_envs = make_vec_env(env_id, n_envs=10)
# budget = 100_000

# log_dir = "data/save/"
# os.makedirs(log_dir, exist_ok=True)

# file_name = "ppo_" + env_id
# log_file_name = log_dir + file_name

# N_TRIALS = 200  # Maximum number of trials
# N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
# N_EVALUATIONS = 2  # Number of evaluations during training
# N_JOBS = 10 # Number of parallel jobs
# N_TIMESTEPS = int(1e5)  # Training budget
# EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
# N_EVAL_ENVS = 5
# N_EVAL_EPISODES = 10
# TIMEOUT = int(60 * 15)  # 15 minutes

# ENV_ID = "LunarLander-v2"

# DEFAULT_HYPERPARAMS = {
#     "policy": "MlpPolicy",
#     "env": ENV_ID,
# }

# def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
#     """Sampler for PPO hyperparameters."""
#     # Discount factor between 0.9 and 0.9999
#     gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
#     max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 8.0, log=True)
#     # 8, 16, 32, ... 1024
#     n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

#     ### YOUR CODE HERE
#     # TODO:
#     # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
#     # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
#     # - define the activation function search space ["tanh", "relu"]
#     learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
#     net_arch = trial.suggest_categorical("archi", ["tiny", "small","medium","big"])
#     activation_fn = trial.suggest_categorical("activ fun", ["tanh", "relu"])

#     ### END OF YOUR CODE

#     # Display true values
#     trial.set_user_attr("gamma_", gamma)
#     trial.set_user_attr("n_steps", n_steps)

#     net_arch = {
#         "tiny": [32],
#         "small": [64, 64],
#         "medium": [256, 256],
#         "big": [400, 300],
#         # Uncomment for tuning HER
#         # "large": [256, 256, 256],
#         # "verybig": [512, 512, 512],
#     }[net_arch]

#     activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

#     return {
#         "n_steps": n_steps,
#         "gamma": gamma,
#         "learning_rate": learning_rate,
#         "max_grad_norm": max_grad_norm,
#         "policy_kwargs": {
#             "net_arch": net_arch,
#             "activation_fn": activation_fn,
#         },
#     }

# class TrialEvalCallback(EvalCallback):
#     """Callback used for evaluating and reporting a trial."""

#     def __init__(
#         self,
#         eval_env: gym.Env,
#         trial: optuna.Trial,
#         n_eval_episodes: int = 10,
#         eval_freq: int = 10000,
#         deterministic: bool = True,
#         verbose: int = 0,
#     ):

#         super().__init__(
#             eval_env=eval_env,
#             n_eval_episodes=n_eval_episodes,
#             eval_freq=eval_freq,
#             deterministic=deterministic,
#             verbose=verbose,
#         )
#         self.trial = trial
#         self.eval_idx = 0
#         self.is_pruned = False

#     def _on_step(self) -> bool:
#         if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
#             super()._on_step()
#             self.eval_idx += 1
#             self.trial.report(self.last_mean_reward, self.eval_idx)
#             # Prune trial if need
#             if self.trial.should_prune():
#                 self.is_pruned = True
#                 return False
#         return True

# def objective(trial: optuna.Trial) -> float:

#     kwargs = DEFAULT_HYPERPARAMS.copy()

#     # Sample hyperparameters and update the keyword arguments
#     new_dict = sample_ppo_params(trial)
#     kwargs.update(new_dict)

#     # Create the RL model
#     model = PPO(**kwargs)
#     # Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`

#     # Create the callback that will periodically evaluate
#     # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
#     eval_callback = TrialEvalCallback(eval_envs, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ)

#     ### END OF YOUR CODE

#     nan_encountered = False
#     try:
#         # Train the model
#         model.learn(N_TIMESTEPS, callback=eval_callback)
#     except AssertionError as e:
#         # Sometimes, random hyperparams can generate NaN
#         print(e)
#         nan_encountered = True
#     finally:
#         # Free memory
#         model.env.close()
#         eval_envs.close()

#     # Tell the optimizer that the trial failed
#     if nan_encountered:
#         return float("nan")

#     if eval_callback.is_pruned:
#         raise optuna.exceptions.TrialPruned()

#     return eval_callback.last_mean_reward

# # Set pytorch num threads to 1 for faster training
# th.set_num_threads(1)
# # Select the sampler, can be random, TPESampler, CMAES, ...
# sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# # Do not prune before 1/3 of the max budget is used
# pruner = MedianPruner(
#     n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
# )
# # Create the study and start the hyperparameter optimization
# study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

# try:
#     study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
# except KeyboardInterrupt:
#     pass
# except (AssertionError, ValueError) as e: 
#      # Sometimes, random hyperparams can generate NaN 
#      print(e)

# print("Number of finished trials: ", len(study.trials))

# print("Best trial:")
# trial = study.best_trial

# print(f"  Value: {trial.value}")

# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# print("  User attrs:")
# for key, value in trial.user_attrs.items():
#     print(f"    {key}: {value}")

# # Write report
# study.trials_dataframe().to_csv("study_results_ppo_cartpole.csv")

In [25]:
def linear_schedule(initial_value):
    """
    Linear learning rate schedule.
    :param initial_value: (float or str)
    :return: (function)
    """
    if isinstance(initial_value, str):
        initial_value = float(initial_value)

    def func(progress):
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress: (float)
        :return: (float)
        """
        return (progress/8) * initial_value

    return func

In [28]:
env = gym.make("LunarLander-v2")
env = make_vec_env("LunarLander-v2", n_envs=8)
#newEnv = Monitor(env, "logs/log.txt")

model = PPO("MlpPolicy", env, verbose=1,  gamma=0.99, 
                                          max_grad_norm=5, 
                                          learning_rate=linear_schedule(0.001), 
                                          n_steps=32, 
                                          ent_coef=0.00001,
                                          policy_kwargs={"net_arch":[256, 128], 
                                                         "activation_fn":nn.ReLU}
                                          )

model.learn(total_timesteps=400_000)

Streaming output truncated to the last 5000 lines.
|    value_loss           | 68           |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 294          |
|    ep_rew_mean          | 70.9         |
| time/                   |              |
|    fps                  | 1182         |
|    iterations           | 1326         |
|    time_elapsed         | 287          |
|    total_timesteps      | 339456       |
| train/                  |              |
|    approx_kl            | 0.0098409075 |
|    clip_fraction        | 0.0883       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.788       |
|    explained_variance   | 0.953        |
|    learning_rate        | 0.000152     |
|    loss                 | 8.32         |
|    n_updates            | 13250        |
|    policy_gradient_loss | -0.00611     |
|    value_loss           | 22           |
---

In [20]:
  model.learn(total_timesteps=200_000)

Streaming output truncated to the last 5000 lines.
|    value_loss           | 47.5        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 218         |
|    ep_rew_mean          | 236         |
| time/                   |             |
|    fps                  | 932         |
|    iterations           | 545         |
|    time_elapsed         | 149         |
|    total_timesteps      | 139520      |
| train/                  |             |
|    approx_kl            | 0.022839338 |
|    clip_fraction        | 0.0613      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.468      |
|    explained_variance   | 0.972       |
|    learning_rate        | 0.0005      |
|    loss                 | 4.3         |
|    n_updates            | 26940       |
|    policy_gradient_loss | 0.00618     |
|    value_loss           | 44.6        |
-------------------------

In [21]:
testing = gym.make("LunarLander-v2")
tst = evaluate_policy(model, testing, return_episode_rewards=True)

In [22]:
from numpy import mean, std
rewards = tst[0]
print( "min:",min(rewards) , "max:",max(rewards), "mean:", mean(rewards), "std:", std(rewards) )

min: 231.15787403862828 max: 277.02416326300744 mean: 257.10903057633243 std: 14.572908416558592


In [24]:
model.save("ppo3_lunar")

In [ ]:
#min: -199.12698238529265 max: 280.0450666236803 mean: 162.34149765391277 std: 141.96902438817605
#min: 44.9088707882911 max: 287.1322615749955 mean: 199.38892376567946 std: 80.09959643588611
#min: 114.74287615103822 max: 288.7991626106086 mean: 252.4218718689473 std: 49.329605053161444
#
#

In [ ]:
# # Viz
# !apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

In [ ]:
# # Set up fake display; otherwise rendering will fail
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

# import base64
# from pathlib import Path

# from IPython import display as ipythondisplay

# def show_videos(video_path='', prefix=''):
#   """
#   Taken from https://github.com/eleurent/highway-env

#   :param video_path: (str) Path to the folder containing videos
#   :param prefix: (str) Filter the video, showing only the only starting with this prefix
#   """
#   html = []
#   for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
#       video_b64 = base64.b64encode(mp4.read_bytes())
#       html.append('''<video alt="{}" autoplay 
#                     loop controls style="height: 400px;">
#                     <source src="data:video/mp4;base64,{}" type="video/mp4" />
#                 </video>'''.format(mp4, video_b64.decode('ascii')))
#   ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

# def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
#   """
#   :param env_id: (str)
#   :param model: (RL model)
#   :param video_length: (int)
#   :param prefix: (str)
#   :param video_folder: (str)
#   """
#   eval_env = DummyVecEnv([lambda: gym.make(env_id)])
#   # Start the video at step=0 and record 500 steps
#   eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
#                               record_video_trigger=lambda step: step == 0, video_length=video_length,
#                               name_prefix=prefix)

#   obs = eval_env.reset()
#   for _ in range(video_length):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, _, _, _ = eval_env.step(action)

#   # Close the video recorder
#   eval_env.close()

In [ ]:
# record_video('LunarLander-v2', model, video_length=500, prefix='ppo-moon')

In [ ]:
# show_videos('videos', prefix='ppo-moon')